# PostgreSQL

In [1]:
import psycopg2
conn = psycopg2.connect(database="test", user="pi", password="")

In [2]:
cur = conn.cursor()

## Check Schema

In [3]:
conn.rollback()

In [4]:
cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema='public' ORDER BY table_name""")
[x for x in cur.fetchall()]

[('joineddate',), ('people',), ('posts',), ('user_comments',), ('users',)]

In [7]:
cur.description

(Column(name='table_name', type_code=1043, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),)

## Create tables

In [ ]:
# user_comments table
cur.execute("""
CREATE TABLE user_comments(
    id integer PRIMARY KEY,
    username text,
    posted_datetime timestamp,
    comments text
)
""")
conn.commit()

In [ ]:
#posts table
cur.execute("""
CREATE TABLE posts(
    username VARCHAR (255) PRIMARY KEY,
    numposts integer
)
""")
conn.commit()

In [ ]:
#joinedDate table
cur.execute("""
CREATE TABLE joinedDate(
    username VARCHAR (255) PRIMARY KEY,
    joined_date timestamp
)
""")
conn.commit()

In [ ]:
#users table
cur.execute("""
CREATE TABLE users(
    username VARCHAR (255) PRIMARY KEY,
    numposts integer,
    joined timestamp
)
""")
conn.commit()

## Load data

In [ ]:
conn = psycopg2.connect(database="test", user="pi", password="")
cur = conn.cursor()
with open('comma_delim_clean.csv', 'r') as f:
    next(f)  # Skip the header row.
    cur.copy_from(f, 'testEight', sep=',')
    
conn.commit()

In [ ]:
conn = psycopg2.connect(database="test", user="pi", password="")
cur = conn.cursor()
with open('users.csv', 'r') as f:
    next(f)  # Skip the header row.
    cur.copy_from(f, 'users', sep=',')
    
conn.commit()

In [ ]:
# build posts table from unique usernames and number of posts
cur.execute('INSERT INTO posts(username, numposts) SELECT username, count(DISTINCT comments) AS num_posts FROM user_comments GROUP BY username')
conn.commit()

In [ ]:
# build joined table from unique usernames, post_counts, and first post date
# cur.execute("""INSERT INTO users SELECT p.username, p.numposts, j.joined_date FROM posts p INNER JOIN joinedDate j ON p.username = j.username ORDER BY p.numposts DESC""")
# conn.commit()

In [ ]:
# build users table from unique usernames, post_counts, and first post date
cur.execute("""INSERT INTO users SELECT p.username, p.numposts, j.joined_date FROM posts p INNER JOIN joinedDate j ON p.username = j.username ORDER BY p.numposts DESC""")
conn.commit()

In [ ]:
cur.execute("""ALTER TABLE users ADD COLUMN is_vip boolean""")
conn.commit()

In [84]:
cur.execute("""UPDATE users SET is_vip = False""")
conn.commit()

### Queries

---
**Question 3**

> You’ve cleaned and imported the file above into the database successfully. Great! The data is stored in a table called user_comments. Now you want to list the top-10 most prolific posters, by username. Write a query that produces this result.

---

In [4]:
cur.execute('SELECT username, count(DISTINCT comments) AS num_posts FROM user_comments GROUP BY username ORDER BY num_posts DESC LIMIT 10')

In [6]:
[x for x in cur.fetchall()]

[('tptacek', 6486),
 ('jacquesm', 4400),
 ('eru', 4207),
 ('pjmlp', 3400),
 ('pg', 2803),
 ('wmf', 2773),
 ('jrockway', 2617),
 ('Tichy', 2588),
 ('gaius', 2569),
 ('icebraining', 2361)]

---

**Question 4**

> There’s another table in your database called users that has the following columns:
 - username
 - name
 - is_vip
 - joined_datetime
>
> Write a query that updates the users table so that only the top-10 posters have a value for is_vip.

---

In [110]:
cur.execute("""UPDATE users SET is_vip = NOT is_vip WHERE username IN (SELECT username FROM users ORDER BY numposts DESC LIMIT 10)""")
conn.commit()

In [117]:
# check is_vip col

In [7]:
cur.execute("""SELECT * FROM users GROUP BY username ORDER BY numposts DESC LIMIT 25 """)

In [8]:
[x for x in cur.fetchall()]

[('tptacek', 6486, datetime.datetime(2007, 12, 10, 1, 4, 19), True),
 ('jacquesm', 4400, datetime.datetime(2009, 1, 30, 22, 30, 52), True),
 ('eru', 4207, datetime.datetime(2007, 8, 16, 16, 3, 6), True),
 ('pjmlp', 3400, datetime.datetime(2011, 10, 9, 12, 33, 54), True),
 ('pg', 2803, datetime.datetime(2006, 10, 9, 19, 52, 45), True),
 ('wmf', 2773, datetime.datetime(2007, 8, 21, 20, 52, 39), True),
 ('jrockway', 2617, datetime.datetime(2007, 5, 13, 7, 26, 39), True),
 ('Tichy', 2588, datetime.datetime(2007, 3, 31, 22, 12, 20), True),
 ('gaius', 2569, datetime.datetime(2008, 5, 29, 17, 18, 15), True),
 ('icebraining', 2361, datetime.datetime(2011, 6, 20, 10, 26, 6), True),
 ('rbanffy', 2349, datetime.datetime(2008, 4, 12, 14, 31, 4), False),
 ('StavrosK', 2321, datetime.datetime(2010, 7, 17, 19, 2, 25), False),
 ('sp332', 2248, datetime.datetime(2009, 4, 29, 15, 54, 22), False),
 ('JoeAltmaier', 2182, datetime.datetime(2009, 10, 20, 15, 20, 52), False),
 ('mikeash', 2082, datetime.date

---

- **Question 5**

> Using both the users and user_comments table, write a query to calculate what percentage of comments were made in the first 30  > days of the users account.
---

In [15]:
# Total time user has been posting
cur.execute("""SELECT u.username, (MAX(c.posted_datetime) - u.joined) FROM users u, user_comments c WHERE u.username='tptacek' GROUP BY u.username""")

In [16]:
[x for x in cur.fetchall()]

[('tptacek', datetime.timedelta(3885, 29878))]

In [33]:
# Needed date range (user, first post, 30 days after first post)
cur.execute("""SELECT u.username, u.joined, (u.joined + interval '30' day) FROM users u, user_comments c WHERE u.username='tptacek' GROUP BY u.username""")

In [34]:
[x for x in cur.fetchall()]

[('tptacek',
  datetime.datetime(2007, 12, 10, 1, 4, 19),
  datetime.datetime(2008, 1, 9, 1, 4, 19))]

In [44]:
# Number of posts in this range (first 30 days)
cur.execute("""
SELECT COUNT(c.comments), u.username
FROM user_comments c, users u
WHERE 
u.username = 'tptacek'
AND c.posted_datetime >= u.joined 
AND c.posted_datetime <=(u.joined + interval '30' day)
GROUP BY u.username
""")

In [45]:
[x for x in cur.fetchall()]

[(1374, 'tptacek')]

In [53]:
# Percentage of posts in this range (first 30 days) out of total
cur.execute("""
SELECT u.username, (COUNT(u.numposts) / COUNT(c.comments)) * 100
FROM user_comments c, users u
WHERE 
u.username = 'tptacek'
AND c.posted_datetime >= u.joined 
AND c.posted_datetime <=(u.joined + interval '30' day)
GROUP BY u.username
""")

In [54]:
[x for x in cur.fetchall()]

[('tptacek', 100)]

---
---
---

In [83]:
cur.execute("""
SELECT COUNT(c.comments)
FROM user_comments c, users u
WHERE 
u.username = 'tptacek'
AND c.posted_datetime >= u.joined 
AND c.posted_datetime <=(u.joined + interval '30' day)
GROUP BY u.username
""")
first_month = cur.fetchone()

In [84]:
total = cur.execute("""
SELECT numposts
FROM users
WHERE 
username = 'tptacek'
""")
total = cur.fetchone()

In [87]:
first_month[0] / total[0] * 100

21.1840888066605

In [59]:
conn.rollback()

In [ ]:
# for row in cur.execute("""SELECT * FROM users"""):
#     total_posts = posts
#     first_30_range = joined + 30d
#     first_30_posts = posts IN first_30_range
#     answer = (COUNT(first_30_posts) / COUNT(total_posts)) * 100